# Simulating Process Flow

# Stages of Manufacturing

## 1. [Raw Material Inventory Store](#first-bullet)

## 2. [Classifier](#second-bullet)

## 3. [Pre-Finish Inventory Store](#third-bullet)

## 4. [Pre-Finish Operation](#fourth-bullet)

## 5. [Pack Inventory Store](#fifth-bullet)

## 6. [Packaging](#sixth-bullet)

![process_flow_diagram](misc/process_flow_diagram.svg)

# TODO:
- **RMI Store**
    - order_drums(): sort drums as dictated by work orders 
- **Classifier**
    - ~~check_pfi_bins(): ensure >5 empty bins available in PFI Store~~
- **PFI Store**
    - assign_drums(): ensure that a given drum maintains the same color & size through a given work order
- **PFO Cell**
    - ~~\_\_repr\_\_(): code proper representation of PFO Cell~~
    - ~~assign_tanks(): determine number of tanks for a given site and initialize them~~
- **PI Cell**
    ~~- assign_drums():~~ 
- **Packaging Cell**
- **Work Orders**
- ~~Finish defining packing cell:~~
    - ~~Assigning machines & properties~~
    - ~~Defining bagging & boxing machines~~
    - ~~Determining process rates~~
- ~~Replace `location` with `plant_id`; update CSVs accordingly~~
- Define how Work Orders are loaded and processed
- Code the overarching simulation process
- Refactoring:
    - ~~Define super/subclass relationship for drums~~
    - ~~Define super/subclass relationship for workcells?~~
    - Pass queue info as dict instead of dataframe
- Thoroughly comment code cells
- Add markdown to illustrate important concepts; break up code cells as necessary
- Add visualizations
- Flowchart Edit Notes:
    - Work orders dictate RMI Store drum order as well as PI Store order.
    - Number and label each work cell for reference in the document.


In [49]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from modules.storage_cell import storage_cell
from modules.processing_cell import processing_cell
from modules.facility import facility

In [50]:
detroit = facility('DETROITMI')

Initializing rmi Cell
Initializing classifier Cell
Initializing pfi Cell
Initializing pfo Cell
Initializing pi Cell
Initializing packaging Cell


In [51]:
rmi = detroit.rmi
cfr = detroit.cfr
pfi = detroit.pfi
pfo = detroit.pfo
pis = detroit.pis
pck = detroit.pck

In [23]:
time = 0
transfer = pd.DataFrame(
    {
        'jb_color':['Coloring Agent1', 'Coloring Agent18'],
        'amount':[45000, 250000]
    }
)
rmi.load_drums(transfer, time=time)

In [24]:
while len(cfr.avail_mach)>0: 
    cfr_in = next(rmi.unload_drums())
    cfr_time = cfr.load_machines(**cfr_in)
    print("Classifier Time: {0}".format(cfr_time))
time += cfr_time
print("Global Time: {0}".format(time))

Classifier Time: 87.71929824561404
Global Time: 87.71929824561404


In [25]:
while len(pfi.empty_drums)>4:
    pfi_cap = min([x.capacity for x in pfi.empty_drums])
    cfr_out = next(cfr.unload_machines(pfi_cap))
    pfi.load_drums(cfr_out, time=time)

In [26]:
flavors = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15']
flavor = (flavor for flavor in flavors)

In [34]:
while len(pfo.avail_mach)>0:
    pfo_in = next(pfi.unload_drums())
    pfo_in['jb_flavor'] = next(flavor)
    pfo_time = pfo.load_machines(**pfo_in)
    print("PFO Time: {0}".format(pfo_time))
time += pfo_time
print("Global Time: {0}".format(time))

PFO Time: 8.563935926968188
PFO Time: 8.842486755881408
Global Time: 118.51159518324468


In [35]:
while len(pis.empty_drums)>0:
    pis_cap = min([x.capacity for x in pis.empty_drums])
    pfo_out = next(pfo.unload_machines(pis_cap))
    pis.load_drums(pfo_out, time=time)

In [ ]:
pis.full_drums

In [36]:
package_types = ['Box', 'Box', 'Box', 'Bag', 'Bag']
package_type = (pack for pack in package_types)

In [37]:
while len(pck.avail_mach)>0:
    pck_in = next(pis.unload_drums())
    pck_in['package_type'] = next(package_type)
    pck_time = pck.load_machines(**pck_in)
    print("Packing Time: {0}".format(pck_time))
time += pck_time
print("Global Time: {0}".format(time))

Packing Time: 2.795368862764303
Packing Time: 2.8383066593290085
Global Time: 121.34990184257369


In [ ]:
pck_in = next(pis.unload_drums())

In [ ]:
pck_in

In [ ]:
pck_in['package_type'] = 'Box'

In [ ]:
pck_in

In [ ]:
pck_time = pck.load_machines(**pck_in)

In [ ]:
pck.machines[0].queue

In [ ]:
pck_time

In [57]:
time=0
flavors = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15']
flavor = (flavor for flavor in flavors)

package_types = ['Box', 'Box', 'Box', 'Bag', 'Bag']
package_type = (pack for pack in package_types)

while len(pis.empty_drums)!=0:
    pck_in = next(pis.unload_drums())
    pck_in['package_type'] = next(package_type)
    pck_time = pck.load_machines(**pck_in)
    print("Packing Time: {0}".format(pck_time))
    time += pck_time
    print("Global Time: {0}".format(time))

    while len(pfo.avail_mach)!=0:
        pis_cap = min([x.capacity for x in pis.empty_drums])
        pfo_out = next(pfo.unload_machines(pis_cap))
        pis.load_drums(pfo_out, time=time)
        
        while len(pfi.empty_drums)!=0:
            pfo_in = next(pfi.unload_drums())
            pfo_in['jb_flavor'] = next(flavor)
            pfo_time = pfo.load_machines(**pfo_in)
            print("PFO Time: {0}".format(pfo_time))
            time += pfo_time
            print("Global Time: {0}".format(time))

            while len(cfr.avail_mach)!=0:
                pfi_cap = min([x.capacity for x in pfi.empty_drums])
                cfr_out = next(cfr.unload_machines(pfi_cap))
                pfi.load_drums(cfr_out, time=time)
                
                while len(rmi.empty_drums)!=0:
                    cfr_in = next(rmi.unload_drums())
                    cfr_time = cfr.load_machines(**cfr_in)
                    print("Classifier Time: {0}".format(cfr_time))
                    time += cfr_time
                    print("Global Time: {0}".format(time))
                else:
                    break
            else:
                break
        else:
            break
    else:
        break

SyntaxError: invalid syntax (<ipython-input-57-7ba26eca4b8c>, line 11)

In [53]:
cfr_out

,jb_color,jb_size,percentage,amount
100,Coloring Agent21,S3,20,10000
101,Coloring Agent21,S1,26,10000
102,Coloring Agent21,S2,20,10000
103,Coloring Agent21,S5,16,10000
104,Coloring Agent21,S4,18,10000


## 1. Raw Material Inventory (RMI) Store <a class="anchor" id="first-bullet"></a>
Several drums act as an inventory store for raw material. The number of drums and their capacities varies by site. Drums can only hold jelly beans with the same color (many-to-one relationship between drums and jelly beans of a given color). One RMI drum can be released at a time into the classifier.

Rules:
1. Order of release from RMI drums is dictated by workorders
2. If there are two RMI drums with the same color at a given location, start emptying the drum with the lower equipment number first
3. Releasing jelly beans from a drum is only allowed if there are at least 5 pre-finished inventory bins empty and available (the next step, classification, is a continuous process which fills pre-finished inventory bins)

In [ ]:
detroit_rmi = storage_cell('rmi', num_drums=40, facility='DETROITMI')

In [ ]:
detroit_rmi.drums

In [ ]:
detroit_rmi.full_drums

In [ ]:
detroit_rmi.empty_drums

In [ ]:
pre_prod_move = pd.DataFrame(
    {
        'Color':['Coloring Agent5', 'Coloring Agent2'],
        'Rem':[176427, 161034]
    }
)

In [ ]:
detroit_rmi.fill_drums(pre_prod_move)

In [ ]:
detroit_rmi.full_drums

In [ ]:
detroit_rmi.empty_drums

In [ ]:
detroit_rmi.fill_drums(pre_prod_move)

## 2. Classifier <a class="anchor" id="second-bullet"></a>

The RMI drums contain jelly beans of the same color, but of varying size. RMI drums are emptied one by one into the classifier, which sorts jelly beans into 5 distinct sizes. Percentage split of each jelly bean size for a given color is provided with the problem statement.

In [ ]:
detroit_classifier = processing_cell('classifier', num_machines=1, facility='DETROITMI')

In [ ]:
detroit_classifier.load_machines(300000, jb_color='Coloring Agent1')

In [ ]:
detroit_classifier.empty_machines(10000)

In [ ]:
detroit_classifier.machines

In [ ]:
detroit_classifier.machines[0].queue

In [ ]:
detroit_classifier.empty_machines(10000)
detroit_classifier.machines[0].queue

In [ ]:
detroit_classifier.empty_machines(10000)
detroit_classifier.machines[0].queue

In [ ]:
detroit_classifier.empty_machines(10000)
detroit_classifier.machines[0].queue

In [ ]:
detroit_classifier.empty_machines(10000)
detroit_classifier.machines[0].queue

In [ ]:
detroit_classifier.empty_machines(10000)
detroit_classifier.machines[0].queue

## 3. Pre-Finish Inventory (PFI) Store <a class="anchor" id="third-bullet"></a>

Jelly beans sorted by the classifier are stored in PFI drums. Drums are assigned a specific jelly bean size and color to avoid contamination (the PFI drums are reused once the drums are emptied into the Pre-Finish Operation). As mentioned earlier, 5 PFI drums (one for each size) must be empty before an RMI drum can be unloaded (thus starting the classification process).

Rules:
1. The classification process simultaneously splits the jelly beans according to the given ratios directly into the PFI drums
2. Fundamentally any PFI drum can be filled with any size of jelly bean; however, once the assignment is made for a given work order, the PFI drum must only hand jelly beans of that size & color until the work order is completely processed
3. PFI drums can only be filled to 95% capacity
4. PFI drums are only available to be emptied into the Pre-Finish Operation once filling has stopped

In [ ]:
detroit_pfi = storage_cell('pfi', num_drums=15, facility='DETROITMI')
detroit_pfi.drums

In [ ]:
detroit_pfi.empty_drums

In [ ]:
detroit_pfi.drums[0].capacity

In [ ]:
detroit_pfi.drums[0].contents

In [ ]:
detroit_pfi.fill_drums(detroit_classifier.empty_machines(10000))

In [ ]:
detroit_pfi.drums[0].contents

In [ ]:
detroit_pfi.drums[0].jb_size

In [ ]:
detroit_pfi.drums[0].jb_color

In [ ]:
detroit_pfi.full_drums

In [ ]:
detroit_pfi.empty_drums

## 4. Pre-Finish Operation (PFO) <a class="anchor" id="fourth-bullet"></a>

PFI drums are now emptied into a tank which applies flavoring to jelly beans of a given color and size (i.e., the tank can only hold 1 unique color-size-flavor combinations at any given time). The number of tanks, tank capacity, and processing rate are given for each facility as part of the problem statement.

Rules:
1. If there is more than 1 PFO tank, assume the rate for each tank is the same.
2. When flavors are changed, there is a change-over duration of 5 minutes.
3. There is no change-over duration between size changes.

In [ ]:
detroit_pfo = processing_cell('pfo', num_machines=2, facility='DETROITMI')

In [ ]:
detroit_pfo

In [ ]:
detroit_pfo.machines

In [ ]:
detroit_pfo.load_machines(10000, jb_color='Coloring Agent1', jb_size='S1', jb_flavor='F1')

In [ ]:
pfo0 = detroit_pfo.machines[0]

In [ ]:
pfo0.available

In [ ]:
pfo0.queue

In [ ]:
pfo0.unload(10000)

In [ ]:
pfo0.jb_color

In [ ]:
pfo0.available

## 5. Pack Inventory (PI) Store <a class="anchor" id="fourth-bullet"></a>

Once the PFO is complete, flavored jelly beans are staged in PI drums. Each PI drum contains fully differentiated product: jelly beans grouped by color, size, and flavor. The number of drums and their capacities vary by site and are given within the problem statement.

Rules:
1. Release of jelly beans into the PFO follows a 'FIFO' policy by default (other policies are allowed but must be explicitly specified).
2. Jelly beans flow continuously from the PFI store through the PFO into the PI store. It can be assumed that any PFO can feed any PI drum.
3. To avoid overflow, PI drums are filled to 95%.
4. PI drums can only be emptied into the Packaging operation once they have been disengaged from the PFO.
5. The lowest PI Drum number is filled up first.
6. Jelly bean colors, flavors, and sizes cannot be mixed in a given drum.
7. Only one PI drum can be emptied into the Packaging operation at a time.
8. The quantity of jelly beans released into the Packaging operation will be determined by the Work Orders.

In [ ]:
detroit_pi = storage_cell('pi', num_drums=8, facility='DETROITMI')

In [ ]:
detroit_pi.drums

In [ ]:
drum = detroit_pi.drums[1]
drum.contents

In [ ]:
drum.capacity

In [ ]:
detroit_pi.full_drums

In [ ]:
detroit_pi.empty_drums

## 6. Packaging Operation <a class="anchor" id="sixth-bullet"></a>

PI drums are emptied for packaging either a bag or a box; i.e., a drum can only be emptied to feed a bag or a box.

Rules:
1. The bagging and box lines cannot be run simultaneously.
2. The box line takes precedence over the bagging line.
3. Orders for boxes are fulfilled prior to orders for bags.
4. All excess material is to be stored in bags.

In [ ]:
detroit_packaging = processing_cell(
    'packaging',
    facility='DETROITMI',
    bagging_machines=1, 
    boxing_machines=1
)

In [ ]:
detroit_packaging.machines

In [ ]:
detroit_packaging.load_machines(
    10000, 
    jb_color='Coloring Agent1', 
    jb_size='S1',
    packaging_type='box'
)

In [ ]:
detroit_packaging.empty_machines(2500)

## Workorder Processing

## Testing & Validation

### Facility Initialization